# 惠聚电商平台用户品类偏好与消费行为深度分析

## 阶段一：数据理解与准备

### 1. 项目背景与目标

**背景**：本项目模拟2022年腾讯旗下新电商平台“惠聚”的品类战略分析。数据来源于一份模拟的早期用户行为数据集（基于`walmart.csv`调整，假设背景为中国市场）。

**目标**：
*   理解数据结构和基本特征。
*   进行必要的数据清洗和预处理，为后续的探索性数据分析（EDA）和深入分析做好准备。
*   初步识别数据中可能存在的质量问题或特点。

### 2. 导入必要的库

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 设置matplotlib和seaborn的样式，使图形更美观
plt.style.use('seaborn-v0_8-whitegrid') # 你可以选择其他你喜欢的seaborn样式
sns.set_palette('muted') # 设置调色板

# 配置pandas显示选项，方便查看数据
pd.set_option('display.max_columns', None) # 显示所有列
pd.set_option('display.max_rows', 100)    # 最多显示100行
pd.set_option('display.float_format', lambda x: '%.3f' % x) # 设置浮点数显示格式

### 3. 加载数据

我们将加载位于 `../data/huiju_sales_data_2022_processed.csv` 的数据集。

In [2]:
# 定义数据文件路径
data_path = '../data/huiju_sales_data_2022_processed.csv'

# 加载数据
try:
    df = pd.read_csv(data_path)
    print("数据加载成功！")
except FileNotFoundError:
    print(f"错误：数据文件未在指定路径找到: {data_path}")
    print("请确认文件路径是否正确，以及文件是否存在。")
    df = None # 如果文件未找到，将df设为None

数据加载成功！


### 4. 初步数据探索

对加载的数据进行初步的探索，了解其基本情况。

In [3]:
if df is not None:
    print("数据前5行：")
    display(df.head())

数据前5行：


,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,83.700
1,1000001,P00248942,F,0-17,10,A,2,0,1,152.000
2,1000001,P00087842,F,0-17,10,A,2,0,12,14.220
3,1000001,P00085442,F,0-17,10,A,2,0,12,10.570
4,1000002,P00285442,M,55+,16,C,4+,0,8,79.690


In [4]:
if df is not None:
    print("\n数据基本信息：")
    df.info()


数据基本信息：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   User_ID                     550068 non-null  int64  
 1   Product_ID                  550068 non-null  object 
 2   Gender                      550068 non-null  object 
 3   Age                         550068 non-null  object 
 4   Occupation                  550068 non-null  int64  
 5   City_Category               550068 non-null  object 
 6   Stay_In_Current_City_Years  550068 non-null  object 
 7   Marital_Status              550068 non-null  int64  
 8   Product_Category            550068 non-null  int64  
 9   Purchase                    550068 non-null  float64
dtypes: float64(1), int64(4), object(5)
memory usage: 42.0+ MB


In [5]:
if df is not None:
    print("\n数值型列描述性统计：")
    display(df.describe())


数值型列描述性统计：


,User_ID,Occupation,Marital_Status,Product_Category,Purchase
count,550068.000,550068.000,550068.000,550068.000,550068.000
mean,1003028.842,8.077,0.410,5.404,92.640
std,1727.592,6.523,0.492,3.936,50.231
min,1000001.000,0.000,0.000,1.000,0.120
25%,1001516.000,2.000,0.000,1.000,58.230
50%,1003077.000,7.000,0.000,5.000,80.470
75%,1004478.000,14.000,1.000,8.000,120.540
max,1006040.000,20.000,1.000,20.000,239.610


In [6]:
if df is not None:
    print("\n非数值型列描述性统计：")
    display(df.describe(include=['object', 'category'])) # 或者 include='O'


非数值型列描述性统计：


,Product_ID,Gender,Age,City_Category,Stay_In_Current_City_Years
count,550068,550068,550068,550068,550068
unique,3631,2,7,3,5
top,P00265242,M,26-35,B,1
freq,1880,414259,219587,231173,193821


In [7]:
if df is not None:
    print("\n各列缺失值数量：")
    missing_values = df.isnull().sum()
    missing_percentage = (df.isnull().sum() / len(df)) * 100
    missing_info = pd.DataFrame({'Missing Values': missing_values, 'Percentage (%)': missing_percentage})
    display(missing_info[missing_info['Missing Values'] > 0].sort_values(by='Missing Values', ascending=False))
    if missing_info['Missing Values'].sum() == 0:
        print("数据中没有缺失值。")


各列缺失值数量：


,Missing Values,Percentage (%)


数据中没有缺失值。


### 5. 详细列检查与数据类型调整

接下来，我们将详细检查一些关键列的唯一值，并根据其特性考虑是否需要调整数据类型，以便于后续分析。

#### 5.1 检查分类列的唯一值

了解分类列中具体的取值情况。

In [8]:
if df is not None:
    print("Gender 唯一值:", df['Gender'].unique())
    print("Gender 值计数:\n", df['Gender'].value_counts())

Gender 唯一值: ['F' 'M']
Gender 值计数:
 Gender
M    414259
F    135809
Name: count, dtype: int64


In [9]:
if df is not None:
    print("Age 唯一值:", df['Age'].unique())
    print("Age 值计数:\n", df['Age'].value_counts().sort_index()) # 按索引排序方便查看

Age 唯一值: ['0-17' '55+' '26-35' '46-50' '51-55' '36-45' '18-25']
Age 值计数:
 Age
0-17      15102
18-25     99660
26-35    219587
36-45    110013
46-50     45701
51-55     38501
55+       21504
Name: count, dtype: int64


In [10]:
if df is not None:
    print("Occupation 唯一值:", sorted(df['Occupation'].unique())) # 排序后查看
    print("Occupation 值计数:\n", df['Occupation'].value_counts().sort_index())

Occupation 唯一值: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20)]
Occupation 值计数:
 Occupation
0     69638
1     47426
2     26588
3     17650
4     72308
5     12177
6     20355
7     59133
8      1546
9      6291
10    12930
11    11586
12    31179
13     7728
14    27309
15    12165
16    25371
17    40043
18     6622
19     8461
20    33562
Name: count, dtype: int64


In [11]:
if df is not None:
    print("City_Category 唯一值:", df['City_Category'].unique())
    print("City_Category 值计数:\n", df['City_Category'].value_counts().sort_index())

City_Category 唯一值: ['A' 'C' 'B']
City_Category 值计数:
 City_Category
A    147720
B    231173
C    171175
Name: count, dtype: int64


In [12]:
if df is not None:
    print("Stay_In_Current_City_Years 唯一值:", df['Stay_In_Current_City_Years'].unique())
    print("Stay_In_Current_City_Years 值计数:\n", df['Stay_In_Current_City_Years'].value_counts().sort_index())

Stay_In_Current_City_Years 唯一值: ['2' '4+' '3' '1' '0']
Stay_In_Current_City_Years 值计数:
 Stay_In_Current_City_Years
0      74398
1     193821
2     101838
3      95285
4+     84726
Name: count, dtype: int64


In [13]:
if df is not None:
    print("Marital_Status 唯一值:", df['Marital_Status'].unique())
    print("Marital_Status 值计数:\n", df['Marital_Status'].value_counts().sort_index())

Marital_Status 唯一值: [0 1]
Marital_Status 值计数:
 Marital_Status
0    324731
1    225337
Name: count, dtype: int64


In [14]:
if df is not None:
    print("Product_Category 唯一值数量:", df['Product_Category'].nunique())
    print("Product_Category 唯一值 (部分示例):", sorted(df['Product_Category'].unique())[:20]) # 显示排序后的前20个，如果总数不多则全显示
    print("Product_Category 值计数 (按品类ID排序):\n", df['Product_Category'].value_counts().sort_index())

Product_Category 唯一值数量: 20
Product_Category 唯一值 (部分示例): [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20)]
Product_Category 值计数 (按品类ID排序):
 Product_Category
1     140378
2      23864
3      20213
4      11753
5     150933
6      20466
7       3721
8     113925
9        410
10      5125
11     24287
12      3947
13      5549
14      1523
15      6290
16      9828
17       578
18      3125
19      1603
20      2550
Name: count, dtype: int64


#### 5.2 检查ID类列的唯一值数量

In [15]:
if df is not None:
    print(f"独立用户数量 (User_ID): {df['User_ID'].nunique()}")
    print(f"独立产品数量 (Product_ID): {df['Product_ID'].nunique()}")

独立用户数量 (User_ID): 5891
独立产品数量 (Product_ID): 3631


#### 5.3 数据类型调整思考与计划

根据以上观察：
*   `Gender`, `Age`, `City_Category`, `Stay_In_Current_City_Years`：已经是 `object` 类型，适合表示分类。`Stay_In_Current_City_Years` 中的 '4+' 是一个特殊值，后续如果需要将其视为有序数值，需要进行转换（例如，将 '4+' 替换为 4 或 5）。目前作为分类可以直接使用。
*   `Occupation`, `Marital_Status`, `Product_Category`：目前是 `int64` 类型，但它们实际上代表的是分类信息（职业代码、婚姻状况代码0/1、产品品类代码）。为了更明确地表达它们的分类属性，并可能利用 pandas `category` 类型的一些优势（如内存优化和特定操作），我们可以考虑将它们转换为 `category` 类型。
*   `User_ID`：虽然是 `int64`，但它是一个标识符，不参与数值计算。可以保持 `int64` 或转换为 `object`/`str`，当前保持 `int64` 也可以。
*   `Product_ID`：已经是 `object`，正确。
*   `Purchase`：是 `float64`，正确。

**计划进行的类型转换：**
*   `Occupation` -> `category`
*   `Marital_Status` -> `category`
*   `Product_Category` -> `category`
*   （可选）`Gender` -> `category`
*   （可选）`Age` -> `category` (已经是 `object`，转为 `category` 可以指定顺序，但非必须)
*   （可选）`City_Category` -> `category`
*   （可选）`Stay_In_Current_City_Years` -> `category` (可以处理 '4+' 为一个特定类别)

In [16]:
if df is not None:
    print("转换前数据类型:")
    print(df.dtypes)

    # 核心转换
    df['Occupation'] = df['Occupation'].astype('category')
    df['Marital_Status'] = df['Marital_Status'].astype('category')
    df['Product_Category'] = df['Product_Category'].astype('category')

    # 可选转换 (object 转 category 通常是为了语义更明确或利用category特性)
    df['Gender'] = df['Gender'].astype('category')
    df['Age'] = df['Age'].astype('category') # 注意：直接转category可能不会保留期望的顺序，后续可视化时可能需要排序
    df['City_Category'] = df['City_Category'].astype('category')
    df['Stay_In_Current_City_Years'] = df['Stay_In_Current_City_Years'].astype('category')


    print("\n转换后数据类型:")
    print(df.dtypes)

    # 验证转换后 Age 列的类别顺序 (如果需要特定顺序)
    # 例如，我们可以定义一个期望的年龄段顺序
    age_categories_ordered = ['0-17', '18-25', '26-35', '36-45', '46-50', '51-55', '55+']
    # df['Age'] = pd.Categorical(df['Age'], categories=age_categories_ordered, ordered=True)
    # print("\nAge列设置为有序分类后的信息:")
    # print(df['Age'].dtype)
    # print(df['Age'].head()) # 查看是否按顺序排列 (这不会改变数据本身排列，而是影响排序和一些绘图行为)

    # 对于 Stay_In_Current_City_Years，如果需要排序，也可以类似处理
    # current_city_years_ordered = ['0', '1', '2', '3', '4+']
    # df['Stay_In_Current_City_Years'] = pd.Categorical(df['Stay_In_Current_City_Years'], categories=current_city_years_ordered, ordered=True)
    # print("\nStay_In_Current_City_Years列设置为有序分类后的信息:")
    # print(df['Stay_In_Current_City_Years'].dtype)

    print("\n再次查看数据基本信息:")
    df.info()

转换前数据类型:
User_ID                         int64
Product_ID                     object
Gender                         object
Age                            object
Occupation                      int64
City_Category                  object
Stay_In_Current_City_Years     object
Marital_Status                  int64
Product_Category                int64
Purchase                      float64
dtype: object

转换后数据类型:
User_ID                          int64
Product_ID                      object
Gender                        category
Age                           category
Occupation                    category
City_Category                 category
Stay_In_Current_City_Years    category
Marital_Status                category
Product_Category              category
Purchase                       float64
dtype: object

再次查看数据基本信息:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype   


### 6. 本阶段总结与后续步骤

**数据理解与准备阶段小结：**

1.  **数据加载**：成功加载了 `huiju_sales_data_2022_processed.csv` 数据集，包含 550,068 条交易记录和 10 个特征。
2.  **初步探索**：
    *   `User_ID`: 5891 名独立用户。
    *   `Product_ID`: 3631 种独立商品。
    *   关键人口统计学特征包括 `Gender`, `Age`, `Occupation`, `City_Category`, `Stay_In_Current_City_Years`, `Marital_Status`。
    *   核心交易特征为 `Product_Category` 和 `Purchase`（购买金额）。
    *   数据中**无缺失值**，简化了清洗流程。
3.  **数据类型转换**：
    *   将 `Gender`, `Age`, `Occupation`, `City_Category`, `Stay_In_Current_City_Years`, `Marital_Status`, `Product_Category` 转换为 `category` 类型，以更准确地反映其数据性质并优化内存使用。
    *   其他字段类型（`User_ID`, `Product_ID`, `Purchase`）保持不变，符合其数据含义。

**后续步骤：**

数据现已准备就绪，可以进入下一个阶段——**探索性数据分析 (EDA)**。在下一份 Notebook (`02_exploratory_data_analysis.ipynb`) 中，我们将深入分析各个变量的分布情况，以及它们之间的初步关系，为后续的用户品类偏好分析打下基础。